In [9]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from pymongo import MongoClient
import sqlite3

SQL_DATABASE_PATH = 'my_sql.db'
MONGO_DB_URL = "mongodb://localhost:27017/"
MONGO_DB_NAME = "DE"
MONGO_COLLECTION_NAME = "fx"
sqlite_file = 'my_sql.db'

In [10]:
# SQLite database
def calculate_average_sqlite():

    conn = sqlite3.connect(sqlite_file)
    cur = conn.cursor()

    cur.execute("SELECT pair, AVG(rate) FROM forex_data GROUP BY pair;")

    data = cur.fetchall()

    # Calculate average of all rates
    cur.execute("SELECT AVG(rate) FROM forex_data")
    overall_average = cur.fetchone()[0]

    cur.close()
    conn.close()

    return data, overall_average

In [11]:
# MongoDB
def calculate_average_mongodb():
    client = MongoClient(MONGO_DB_URL)
    db = client[MONGO_DB_NAME]
    collection = db[MONGO_COLLECTION_NAME]

    averages = {}

    for pair in ["EUR,USD", "USD,GBP", "GBP,CAD"]:
        pipeline = [
            {"$match": {"pair": pair}},
            {"$group": {"_id": None, "averageRate": {"$avg": "$rate"}}}
        ]
        result = list(collection.aggregate(pipeline))
        if result:
            average = result[0]["averageRate"]
            averages[pair] = average
        else:
            averages[pair] = None

    # Calculate average of all rates
    pipeline_all = [{"$group": {"_id": None, "overall_average": {"$avg": "$rate"}}}]
    result_all = list(collection.aggregate(pipeline_all))
    overall_average = result_all[0]["overall_average"] if result_all else None

    client.close()
    return averages, overall_average

In [12]:
def main():
    sqlite_averages, sqlite_overall_average = calculate_average_sqlite()
    print("SQLite Averages (pairwise):", sqlite_averages)
    print("SQLite Average (all data points):", sqlite_overall_average)

    mongo_averages, mongo_overall_average = calculate_average_mongodb()
    print("MongoDB Averages (pairwise):", mongo_averages)
    print("MongoDB Average (all data points):", mongo_overall_average)

if __name__ == "__main__":
    main()

SQLite Averages (pairwise): [('EUR,USD', 1.0847482929441774), ('GBP,CAD', 1.7200000000002393), ('USD,GBP', 0.7852517070560839)]
SQLite Average (all data points): 1.1966666666673498
MongoDB Averages (pairwise): {'EUR,USD': 1.08, 'USD,GBP': 0.79, 'GBP,CAD': 1.72}
MongoDB Average (all data points): 1.1966666666666668
